In [1]:
# set the matplotlib backend so figures can be saved in the background
import matplotlib
matplotlib.use("Agg")

# import the necessary packages
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import img_to_array
from keras.utils import to_categorical
import matplotlib.pyplot as plt
import numpy as np
import cv2
import csv

from keras.models import Sequential
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras.layers import Dropout
from keras import backend as K
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

from keras.models import load_model

Using TensorFlow backend.


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 4412028203214940326
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 3164969369
locality {
  bus_id: 1
  links {
  }
}
incarnation: 6977104006297754605
physical_device_desc: "device: 0, name: GeForce GTX 1050, pci bus id: 0000:01:00.0, compute capability: 6.1"
]


In [2]:
class NetCNN:
    @staticmethod
    def build(width, height, depth, classes):
        # initialize the model
        model = Sequential()
        inputShape = (height, width, depth)
 
        # if we are using "channels first", update the input shape
        if K.image_data_format() == "channels_first":
            inputShape = (depth, height, width)
        # first set of CONV => RELU => POOL layers
        model.add(Conv2D(20, (5, 5), padding="same",input_shape=inputShape))
        model.add(Activation("relu"))
        model.add(Dropout(0.2))
        model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
        # second set of CONV => RELU => POOL layers
        model.add(Conv2D(50, (5, 5), padding="same"))
        model.add(Activation("relu"))
        model.add(Dropout(0.2))
        model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
        # third set of CONV => RELU => POOL layers
        model.add(Conv2D(20, (5, 5), padding="same",input_shape=inputShape))
        model.add(Activation("relu"))
        model.add(Dropout(0.2))
        model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
        # first (and only) set of FC => RELU layers
        model.add(Flatten())
        model.add(Dense(500))
        model.add(Activation("relu"))
        model.add(Dropout(0.3))
        
        # softmax classifier
        model.add(Dense(classes))
        model.add(Activation("softmax"))

        # return the constructed network architecture
        return model

In [3]:
# initialize the data and labels
print("[INFO] loading images...")
data = []
labels = []

with open("..\\data\\data\\csv\\galaxy\\galaxy_label_data_set.csv", 'rt') as csvFile:
    reader = csv.reader(csvFile, delimiter=",")
    # loop over the input images
    for index, row in enumerate(reader):
        if index in range(1,1000):
            # load the image, pre-process it, and store it in the data list
            file = "..\\data\\data\\images\\"+row[0]+".jpg"
            image = cv2.imread(file)
            image = cv2.resize(image, (212, 212))
            image = img_to_array(image)
            data.append(image)
            label = (0 if row[1] == 'smooth' else 1)
            labels.append(label)

# scale the raw pixel intensities to the range [0, 1]
data = np.array(data, dtype="float") / 255.0
labels = np.array(labels)

# partition the data into training and testing splits using 75% of
# the data for training and the remaining 25% for testing
(trainX, testX, trainY, testY) = train_test_split(data, labels, test_size=0.25, random_state=42)

# convert the labels from integers to vectors
trainY = to_categorical(trainY, num_classes=2)
testY = to_categorical(testY, num_classes=2)

# construct the image generator for data augmentation
aug = ImageDataGenerator(rotation_range=30, width_shift_range=0.1,
    height_shift_range=0.1, shear_range=0.2, zoom_range=0.2,
    horizontal_flip=True, fill_mode="nearest")

[INFO] loading images...


In [4]:
EPOCHS = 50
INIT_LR = 1e-3
BS = 25

# initialize the model
print("[INFO] compiling model...")
model = NetCNN.build(width=212, height=212, depth=3, classes=2)
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy"])

# train the network
print("[INFO] training network...")
H = model.fit_generator(aug.flow(trainX, trainY, batch_size=BS),
    validation_data=(testX, testY), steps_per_epoch=len(trainX) // BS,
    epochs=EPOCHS, verbose=1)

# save the model to disk
print("[INFO] serializing network...")
model.save("..\\data\\tp3\\model.hdf5")

# plot the training loss and accuracy
plt.style.use("ggplot")
plt.figure()
N = EPOCHS
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["acc"], label="train_acc")
plt.plot(np.arange(0, N), H.history["val_acc"], label="val_acc")
plt.title("Training Loss and Accuracy on Smooth/Spiral")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig("Graphs\\3c25bs3lr.png")

[INFO] compiling model...
[INFO] training network...
Epoch 1/50
29/29 [==============================] - 11s 379ms/step - loss: 0.6270 - acc: 0.6345 - val_loss: 0.4703 - val_acc: 0.8440
Epoch 2/50
29/29 [==============================] - 8s 266ms/step - loss: 0.5847 - acc: 0.7639 - val_loss: 0.4438 - val_acc: 0.8600
Epoch 3/50
29/29 [==============================] - 8s 264ms/step - loss: 0.5217 - acc: 0.7805 - val_loss: 0.4497 - val_acc: 0.8080
Epoch 4/50
29/29 [==============================] - 8s 265ms/step - loss: 0.4374 - acc: 0.8272 - val_loss: 0.3855 - val_acc: 0.9040
Epoch 5/50
29/29 [==============================] - 8s 266ms/step - loss: 0.3849 - acc: 0.8371 - val_loss: 0.3602 - val_acc: 0.8560
Epoch 6/50
29/29 [==============================] - 8s 262ms/step - loss: 0.4357 - acc: 0.8026 - val_loss: 0.4023 - val_acc: 0.8880
Epoch 7/50
29/29 [==============================] - 8s 269ms/step - loss: 0.3585 - acc: 0.8439 - val_loss: 0.3210 - val_acc: 0.9200
Epoch 8/50
29/29 [====